In [30]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
T = df['Year'].unique().size
df['const']=np.ones((Nobs,1))

#Skalér for bedre fortolkning

df['Price'] = df['Price']/10_000 #(Ændring i ms(%) for en ændring i pris på 10.000)
df['HP'] = df['HP']/10           #(Ændring i ms(%) for en ændring i HP på 10)
df['Range'] = df['Range']/10     #(Ændring i ms(%) for en ændring i rækkevidde på 10)

In [32]:
estimation = OLS.get_coefficients_OLS(df)


model_labels = df['Model'].unique()
#j = 0
#for i in model_labels:
#    j = j
#    print(f'Car {j}: {i}')
#    j = j+1  

repeated_models = np.repeat(model_labels, T)
print(f' models: {len(model_labels)} \n repeated models: {len(repeated_models)}')
      
print(f'\n \n Coefficients:\n{estimation} \ntype: {type(estimation)} \n shape: {estimation.shape} \n\n')



 models: 107 
 repeated models: 1177

 
 Coefficients:
const        -3.628008
Price        -0.000187
Range         0.083073
HP           -0.018426
Chargetime   -0.034479
Segment_B    -0.321690
Segment_C     0.304920
Segment_D    -0.073204
Segment_E    -1.514931
Segment_F    -1.772606
Segment_M    -1.532133
Segment_J    -0.508531
Year_2014     0.482998
Year_2015    -0.377237
Year_2016     0.032114
Year_2017    -0.175466
Year_2018    -0.800639
Year_2019    -1.211335
Year_2020    -1.870289
Year_2021    -2.874557
Year_2022    -3.302745
Year_2023    -3.927324
China        -0.644365
dtype: float64 
type: <class 'pandas.core.series.Series'> 
 shape: (23,) 




In [33]:
#Manipuler for ms = 0 ikke har ssh for at blive væltet.
df_calc = df.copy()
df_calc = pd.get_dummies(df_calc, columns=['Segment'], drop_first=True)
df_calc = pd.get_dummies(df_calc, columns=['Year'], drop_first=True)
df_calc['China'] = (df_calc['Country'] == 'CH').astype(int)

df_calc = fun.straf_0ms(df_calc)

df_dummies = df.copy()
df_dummies = pd.get_dummies(df_dummies, columns=['Segment'], drop_first=True)
df_dummies = pd.get_dummies(df_dummies, columns=['Year'], drop_first=True)
df_dummies['China'] = (df_dummies['Country'] == 'CH').astype(int)


print(f' df_calc shape: {df_calc.shape} \n df_dummies shape: {df_dummies.shape}')


c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i] = -df['Price'][i]*1000


 df_calc shape: (1177, 30) 
 df_dummies shape: (1177, 30)


In [34]:
x = df_calc[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = df_calc[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)

columns = X.columns
print(f' columns: {columns}')

X = np.array(X)
print(f' X shape: {X[:,2:].shape}\n X type {type(X)}\n')

p_j = df_calc['Price']
p_j = np.array(p_j)
print(f' p_j shape: {p_j.shape} \n p_j type: {type(p_j)}\n ')

alpha = estimation['Price']
alpha = np.array(alpha)
beta = estimation
beta = np.array(beta)
print(f' alpha {alpha} \n beta shape {beta.shape}')


 columns: Index(['const', 'Price', 'Range', 'HP', 'Chargetime', 'Segment_B', 'Segment_C',
       'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
       'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018',
       'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
       'China'],
      dtype='object')
 X shape: (1177, 21)
 X type <class 'numpy.ndarray'>

 p_j shape: (1177,) 
 p_j type: <class 'numpy.ndarray'>
 
 alpha -0.00018694188592087522 
 beta shape (23,)


In [35]:
ccp_list = fun.ccp(alpha, beta, X, p_j)

share_j: 1177
 choice probability sum: 0.9999999999999999 
 ccp:[array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.00117641]]), array([[0.00076624]]), array([[0.00041069]])]


In [36]:
model_probability = pd.DataFrame({'Model': repeated_models, 'CCP': ccp_list})
print(f' shape: {model_probability.shape} \n model_probability: {model_probability}')


 shape: (1177, 2) 
 model_probability:      Model                        CCP
0       U5                    [[0.0]]
1       U5                    [[0.0]]
2       U5                    [[0.0]]
3       U5                    [[0.0]]
4       U5                    [[0.0]]
...    ...                        ...
1172  Free                    [[0.0]]
1173  Free                    [[0.0]]
1174  Free                    [[0.0]]
1175  Free                    [[0.0]]
1176  Free  [[4.392115263372249e-05]]

[1177 rows x 2 columns]


In [37]:
probability_ratio = fun.probability_ratio(ccp_list[:10], repeated_models[:10])
print(f'shape: {probability_ratio.shape} \n probability_ratio: {probability_ratio}')

shape: (10, 10) 
 probability_ratio:          U5       U5       U5       U5       U5       U5       U5       U5  \
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]  [[nan]]   
U5  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]   
U5  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]  [[inf]]   

                        U5                      U5  
U5                 [[0.0]]                 [[0.0]] 

c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:36: RuntimeWarning: invalid value encountered in true_divide
  probability_ratio_matrix.iloc[i,j] = ccp[i]/ccp[j]
c:\Users\Mikkel\OneDrive\Offentlig\Dokumenter\GitHub\Bachelor\functions.py:36: RuntimeWarning: divide by zero encountered in true_divide
  probability_ratio_matrix.iloc[i,j] = ccp[i]/ccp[j]


In [38]:
marginal_effects = fun.marginal_effects(ccp_list, repeated_models, columns, estimation)
print(f'shape: {marginal_effects.shape} \n marginal_effects: {marginal_effects}')

shape: (1177, 23) 
 marginal_effects:                            const                       Price  \
U5                      [[-0.0]]                     [[0.0]]   
U5                      [[-0.0]]                     [[0.0]]   
U5                      [[-0.0]]                     [[0.0]]   
U5                      [[-0.0]]                     [[0.0]]   
U5                      [[-0.0]]                     [[0.0]]   
...                          ...                         ...   
Free                    [[-0.0]]                     [[0.0]]   
Free                    [[-0.0]]                     [[0.0]]   
Free                    [[-0.0]]                     [[0.0]]   
Free                    [[-0.0]]                     [[0.0]]   
Free  [[-8.210703105166697e-09]]  [[3.6486520644254584e-06]]   

                           Range                           HP  \
U5                      [[-0.0]]                     [[-0.0]]   
U5                      [[-0.0]]                     [[-0.0]]  

In [39]:
cross_marginal_effects = fun.cross_marginal_effects(ccp_list, estimation)
print(f'shape: {cross_marginal_effects.shape} \n cross_marginal_effects: {cross_marginal_effects}')

(1177, 1177, 22)
shape: (1177, 1177, 22) 
 cross_marginal_effects: [[[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -0.00000000e+00  0.00000000e+00 ...  0.00000000e+0

In [40]:
elasticity = fun.elasticity(ccp_list, repeated_models, columns, estimation, X)
print(f'shape: {elasticity.shape} \n elasticity: {elasticity}')

elasticity shape: 
(1177, 23)
shape: (1177, 23) 
 elasticity:                         const                   Price  \
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
U5     [[-1869.418859208752]]  [[3.3229110309131893]]   
...                       ...                     ...   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free   [[-1869.418859208752]]  [[3.3229110309131893]]   
Free  [[-1869.3367521777004]]  [[3.3227650848306123]]   

                         Range                       HP Chargetime  \
U5    [[-0.37035523076277393]]  [[-1.1722891235561768]]   [[-0.0]]   
U5    [[-0.37035523076277393]]  [[-1.1722891235561768]]   [[-0.0]]   
U5    [[-0.37035523076277393]]  [[-1.172289

In [41]:
cross_elasticity = fun.cross_elasticity(ccp_list, estimation, X, repeated_models)

1177
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 ...

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]]
Change in : U5 
 [[[ 0. -0.  0. ...  0.  0.  0.]]

 [[ 0. -0.  0. ...  0.  0.  0.]]

 [